In [ ]:
import os
import librosa
import numpy as np
import pandas as pd

In [2]:
# Function to extract features from a given audio file
def extract_features(audio_file, target_duration=4, target_sr=44100):

    # Load audio and define target sample rate
    y, sr = librosa.load(audio_file, sr=target_sr)

    # Ensure audio length is the same as the target duration using zero padding
    target_length = int(target_sr * target_duration)
    y = librosa.util.fix_length(y, size=target_length)

    # Normalize amplitude
    y = librosa.util.normalize(y)

    # Define variables for some features
    hop_length = 512
    oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)

    # spectral feature list
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)
    chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
    melspectogram = librosa.feature.melspectrogram(y=y, sr=sr)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    rms = librosa.feature.rms(y=y)
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    flatness = librosa.feature.spectral_flatness(y=y)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    crossing_rate = librosa.feature.zero_crossing_rate(y)

    # rhythm feature list
    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)
    fourier_tempogram = librosa.feature.fourier_tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)

    # Create a dictionary to store the mean value of the features
    features = {
        'chroma_stft': np.mean(chroma_stft),
        'chroma_cqt': np.mean(chroma_cqt),
        'chroma_cens': np.mean(chroma_cens),
        'melspectogram': np.mean(melspectogram),
        'rms': np.mean(rms),
        'centroid': np.mean(centroid),
        'bandwidth': np.mean(bandwidth),
        'contrast': np.mean(contrast),
        'flatness': np.mean(flatness),
        'rolloff': np.mean(rolloff),
        'crossing_rate': np.mean(crossing_rate),
        'tempogram': np.mean(tempogram),
        'fourier_tempogram': np.mean(fourier_tempogram)
    }

    # Add the mean of each MFCC feature to the dictionary
    for i in range(1, 41):
        features[f'mcffs_{i}'] = np.mean(mfccs[i - 1])
    return features

13 mfccs

In [5]:
def extract_13features(audio_file, target_duration=4, target_sr=44100):
    y, sr = librosa.load(audio_file, sr=target_sr)

    target_length = int(target_sr * target_duration)
    y = librosa.util.fix_length(y, size=target_length)
    y = librosa.util.normalize(y)

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    features13 = {}
    
    for i in range(13):
        features13[f'mfccs_{i + 1}'] = np.mean(mfccs[i])

    return features13



In [8]:
def process_data(base_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for folder in os.listdir(base_dir):
        # Ignora diretórios que terminam com '_4sec'
        if folder.endswith('_4sec'):
            continue

        label_list = []
        features_list = []

        fold_dir = os.path.join(base_dir, folder)
        if os.path.isdir(fold_dir):
            for filename in os.listdir(fold_dir):
                file_path = os.path.join(fold_dir, filename)
                if filename.endswith('.wav'):
                    label = filename
                    features = extract_features(file_path)
                    features_list.append(features)
                    label_list.append(label)

            # Cria o DataFrame para cada pasta
            df = pd.DataFrame(features_list)
            df['Label'] = label_list
            
            # Nomeia o arquivo consistentemente
            output_file = os.path.join(output_dir, f'urbansounds_13features_fold{folder}.csv')
            df.to_csv(output_file, index=False)
            print(f"Saved: {output_file}")
            print(df.head())

In [9]:
def main():
    base_dir = "UrbanSound8K/audio/"
    output_dir = "UrbanSound8K/features/"
    process_data(base_dir, output_dir)

if __name__ == '__main__':
    main()


Saved: UrbanSound8K/features/urbansounds_13features_foldfold1.csv
   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.494262    0.568759     0.267266       3.465249  0.068657  2177.483658   
1     0.422076    0.556903     0.270826       3.132792  0.076706  1623.980658   
2     0.575613    0.606710     0.275953       2.257987  0.044251  2615.200756   
3     0.175040    0.524367     0.259533       1.481219  0.043195  1223.789196   
4     0.412279    0.608818     0.264944       0.806509  0.035636  1470.863386   

     bandwidth   contrast  flatness      rolloff  ...    mfccs_5   mfccs_6  \
0  3649.518344  19.685466  0.002088  4745.917969  ... -13.181735  0.722893   
1  2897.277104  21.508354  0.001123  2743.891814  ... -22.950523 -2.278975   
2  4200.855657  18.001559  0.003370  6143.329229  ...  -2.329756  5.860126   
3  1367.391253  16.179755  0.583564  2473.821756  ... -12.256881  6.895666   
4  2296.662162  12.349992  0.353831  3264.932914  ...  -2

c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=345
  warnings.warn(
c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Saved: UrbanSound8K/features/urbansounds_13features_foldfold10.csv
   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.062613    0.554600     0.262214       1.433406  0.036064   443.916662   
1     0.130186    0.579297     0.267485       2.023491  0.054990   741.250803   
2     0.040313    0.548530     0.258703       0.471699  0.020168   329.020390   
3     0.206296    0.600387     0.267217       2.063086  0.073937  1115.613531   
4     0.167365    0.573478     0.264327       3.879928  0.089565  1768.380517   

     bandwidth   contrast  flatness      rolloff  ...    mfccs_5    mfccs_6  \
0   549.756662  11.759230  0.832015  1020.736243  ...  -7.775619   4.292518   
1   852.699195  12.376828  0.730634  1537.033798  ...  -9.076921   4.267955   
2   337.283299  11.662411  0.893353   641.252548  ...  -2.490561   1.280010   
3  1195.485393  11.253914  0.588647  2284.017493  ... -13.796725  11.079891   
4  1632.312511  19.919305  0.528298  3634.555027  .

c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=345
  warnings.warn(
c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Saved: UrbanSound8K/features/urbansounds_13features_foldfold2.csv
   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.410848    0.599665     0.275006       1.618515  0.062880  2315.323931   
1     0.392929    0.587738     0.272814       1.741088  0.062846  2252.724403   
2     0.410774    0.596085     0.276620       1.731248  0.058378  2262.007037   
3     0.392908    0.556547     0.268771       2.688253  0.072644  2154.097000   
4     0.104662    0.499649     0.250140       0.703572  0.021337   633.364388   

     bandwidth   contrast  flatness      rolloff  ...   mfccs_5   mfccs_6  \
0  2083.987909  18.072906  0.000642  3774.239980  ... -3.717161  7.350816   
1  1938.231196  17.514468  0.000307  3473.648947  ... -0.191985  6.899851   
2  1977.702029  17.283199  0.000360  3553.665082  ...  2.913156  6.744619   
3  1830.938122  17.564426  0.000349  3266.493291  ...  5.697710  5.440441   
4   691.084327  16.920681  0.791836  1174.963910  ... -3.76909

...

Saved: UrbanSound8K/features/urbansounds_13features_foldfold7.csv
   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.473349    0.423699     0.252005       7.211914  0.184888  1939.300836   
1     0.509128    0.417532     0.249702       8.005108  0.194816  1941.895823   
2     0.489459    0.424997     0.249331       6.819539  0.180989  2090.856747   
3     0.492910    0.447101     0.251485       6.990344  0.180620  1942.654082   
4     0.507499    0.491998     0.261593       1.303355  0.077784  1856.432076   

     bandwidth   contrast  flatness      rolloff  ...   mfccs_5    mfccs_6  \
0  3242.532292  16.510912  0.001401  3601.038128  ... -4.109747  25.261068   
1  3268.910604  16.209399  0.001693  3578.818359  ... -4.505092  25.072016   
2  3404.391244  16.646178  0.002029  3916.234290  ... -5.411260  23.130854   
3  3270.532335  16.579704  0.001512  3529.385615  ... -4.070893  23.280151   
4  3182.401065  15.435954  0.001516  3218.059188  ... -6.

c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=345
  warnings.warn(


Saved: UrbanSound8K/features/urbansounds_13features_foldfold8.csv
   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.208078    0.517414     0.262911       2.119976  0.033387  1397.956162   
1     0.069300    0.533790     0.264020       1.382705  0.021139   444.913739   
2     0.037289    0.526324     0.255012       0.685282  0.010956   193.335421   
3     0.453469    0.617095     0.278163       2.494233  0.118673  2516.334483   
4     0.464953    0.584813     0.276180       3.275724  0.135394  2624.715888   

     bandwidth   contrast  flatness      rolloff  ...    mfccs_5   mfccs_6  \
0  2304.573587  18.432416  0.363803  2433.189538  ... -11.222441 -9.354464   
1   723.332913  21.141853  0.751068   649.678584  ...  -7.664314 -5.783936   
2   320.915987  18.381143  0.881263   258.835343  ...  -3.857676 -2.426065   
3  2471.973714  15.317669  0.001505  4590.129925  ... -23.182991  7.930934   
4  2602.535122  15.442079  0.001736  4884.541865  ... -26

c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=345
  warnings.warn(
c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Saved: UrbanSound8K/features/urbansounds_13features_foldfold9.csv
   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.515049    0.638425     0.278488       0.753837  0.099621  6900.971452   
1     0.438766    0.612939     0.277053       0.724370  0.096050  6414.897328   
2     0.476709    0.620613     0.276970       0.670799  0.093188  6757.544286   
3     0.521667    0.644786     0.276738       0.637256  0.089636  6892.575977   
4     0.536578    0.638835     0.276182       0.542374  0.078891  6518.401026   

     bandwidth   contrast  flatness       rolloff  ...   mfccs_5    mfccs_6  \
0  4778.398755  17.307878  0.052327  12039.120245  ... -6.161792  12.510046   
1  5116.674017  17.648697  0.020844  12080.251783  ... -1.499629   5.982838   
2  4930.859583  17.540357  0.036628  12136.238111  ... -4.938836   9.615676   
3  4809.401195  17.388613  0.048931  12037.497452  ... -5.884266  12.034477   
4  5018.789984  17.194120  0.036717  12003.169158  .

In [4]:

def main():
    base_dir = "UrbanSound8K/audio/"
    process_data(base_dir)


if __name__ == '__main__':
    main()

Empty DataFrame
Columns: [Label]
Index: []


c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=345
  warnings.warn(
c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.494262    0.568759     0.267266       3.465249  0.068657  2177.483658   
1     0.422076    0.556903     0.270826       3.132792  0.076706  1623.980658   
2     0.575613    0.606710     0.275953       2.257987  0.044251  2615.200756   
3     0.175040    0.524367     0.259533       1.481219  0.043195  1223.789196   
4     0.412279    0.608818     0.264944       0.806509  0.035636  1470.863386   

     bandwidth   contrast  flatness      rolloff  ...    mfccs_5   mfccs_6  \
0  3649.518344  19.685466  0.002088  4745.917969  ... -13.181735  0.722893   
1  2897.277104  21.508354  0.001123  2743.891814  ... -22.950523 -2.278975   
2  4200.855657  18.001559  0.003370  6143.329229  ...  -2.329756  5.860126   
3  1367.391253  16.179755  0.583564  2473.821756  ... -12.256881  6.895666   
4  2296.662162  12.349992  0.353831  3264.932914  ...  -2.311979  1.896574   

     mfccs_7    mfccs_8    mfccs_9   mfccs_1

c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=345
  warnings.warn(
c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.062613    0.554600     0.262214       1.433406  0.036064   443.916662   
1     0.130186    0.579297     0.267485       2.023491  0.054990   741.250803   
2     0.040313    0.548530     0.258703       0.471699  0.020168   329.020390   
3     0.206296    0.600387     0.267217       2.063086  0.073937  1115.613531   
4     0.167365    0.573478     0.264327       3.879928  0.089565  1768.380517   

     bandwidth   contrast  flatness      rolloff  ...    mfccs_5    mfccs_6  \
0   549.756662  11.759230  0.832015  1020.736243  ...  -7.775619   4.292518   
1   852.699195  12.376828  0.730634  1537.033798  ...  -9.076921   4.267955   
2   337.283299  11.662411  0.893353   641.252548  ...  -2.490561   1.280010   
3  1195.485393  11.253914  0.588647  2284.017493  ... -13.796725  11.079891   
4  1632.312511  19.919305  0.528298  3634.555027  ...  -7.457651   9.514107   

     mfccs_7   mfccs_8   mfccs_9  mfcc

...

   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.418725    0.612387     0.278292       1.607309  0.074105  3043.966183   
1     0.505951    0.669624     0.281844       4.061538  0.164876  2781.675372   
2     0.571285    0.647357     0.278443       4.981573  0.197318  2466.199590   
3     0.603959    0.633611     0.276199       5.573277  0.203604  2340.934009   
4     0.608034    0.649573     0.278151       5.326177  0.199163  2355.517437   

     bandwidth   contrast  flatness      rolloff  ...    mfccs_5    mfccs_6  \
0  3294.231853  16.270337  0.001726  5852.599779  ... -18.994778   9.542340   
1  2188.558867  17.932475  0.003873  4399.077361  ... -38.086643  21.989058   
2  1982.693441  18.322601  0.000143  3877.100034  ... -33.659439  14.159001   
3  1963.525652  18.132455  0.000073  3727.054178  ... -43.735199  18.355787   
4  1991.233737  17.804676  0.000106  3728.801800  ... -42.802326  17.422201   

     mfccs_7    mfccs_8    mfccs_9   m

c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=345
  warnings.warn(
c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.350722    0.628669     0.274464       2.201034  0.085976  2199.909282   
1     0.244823    0.483305     0.260922       7.867567  0.185764  3229.853736   
2     0.413127    0.617291     0.274670       2.175758  0.116365  3776.908245   
3     0.264808    0.481658     0.264932       3.444897  0.136996  3508.129253   
4     0.556982    0.549648     0.269304       3.239949  0.143240  2840.936636   

     bandwidth   contrast  flatness      rolloff  ...    mfccs_5    mfccs_6  \
0  2462.985802  16.192979  0.221532  4193.856573  ... -21.145796  12.998729   
1  3684.318422  15.807474  0.006726  6132.032099  ... -22.864326   6.856188   
2  3889.828050  14.676860  0.016938  7172.304263  ... -19.004459  13.270333   
3  3597.622542  16.034722  0.006623  6922.581522  ... -33.639786  16.202541   
4  3153.703887  14.476056  0.005971  5070.039487  ... -11.698614   7.442532   

     mfccs_7    mfccs_8    mfccs_9   m

c:\Users\utilizador\Documents\3 ano\ML2\mlp\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=384 is too large for input signal of length=345
  warnings.warn(


KeyboardInterrupt: 

,chroma_stft,chroma_cqt,chroma_cens,melspectogram,rms,centroid,bandwidth,contrast,flatness,rolloff,...,mcffs_32,mcffs_33,mcffs_34,mcffs_35,mcffs_36,mcffs_37,mcffs_38,mcffs_39,mcffs_40,Label
0,0.494262,0.568759,0.267266,3.465249,0.068657,2177.483658,3649.518344,19.685466,2.088375e-03,4745.917969,...,-2.700133,-0.350788,-0.123943,-0.308717,0.377525,-0.907509,-0.254973,-1.738416,-1.207377,101415-3-0-2.wav
1,0.422076,0.556903,0.270826,3.132792,0.076706,1623.980658,2897.277104,21.508354,1.122673e-03,2743.891814,...,-5.048006,-1.626652,-1.303208,-1.028367,0.470883,-2.959822,-1.439513,-2.282194,-1.155120,101415-3-0-3.wav
2,0.575613,0.606710,0.275953,2.257988,0.044251,2615.200756,4200.855657,18.001559,3.370488e-03,6143.329229,...,-1.015445,0.989107,1.142546,0.407650,-0.166602,-0.859933,-0.637618,-1.657652,-1.639226,101415-3-0-8.wav
3,0.175040,0.524367,0.259533,1.481219,0.043195,1223.789196,1367.391253,16.179755,5.835638e-01,2473.821756,...,2.447487,-1.956081,0.440909,-1.384400,0.585570,-0.601787,-0.864582,-0.896903,-0.161344,102106-3-0-0.wav
4,0.412279,0.608818,0.264944,0.806509,0.035636,1470.863386,2296.662162,12.349992,3.538312e-01,3264.932914,...,0.232175,0.326373,0.610677,0.570734,1.030308,0.507107,0.160745,0.642600,0.364088,102305-6-0-0.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,0.489021,0.532514,0.267923,3.619157,0.131127,1001.743077,856.109048,21.385680,1.188957e-05,1890.115914,...,-6.047721,0.567317,-3.426983,-3.207953,3.002672,-0.099220,-3.515616,2.629431,2.310546,99180-9-0-2.wav
869,0.388119,0.432899,0.256128,8.101979,0.208449,1174.695037,934.756531,22.675136,9.659468e-07,2255.056895,...,-2.078834,0.630212,-7.566489,-4.593991,3.579275,-4.502560,-8.744062,5.975438,8.999461,99180-9-0-36.wav
870,0.413462,0.507484,0.255163,9.242000,0.224102,1190.957197,915.262998,21.871048,6.211397e-07,2320.904806,...,-5.864554,2.098599,-0.605085,-1.797897,2.578238,-0.928515,-2.693557,7.228871,7.933712,99180-9-0-48.wav
871,0.413023,0.482618,0.254830,8.568559,0.214565,1195.251424,924.910023,21.599182,6.669997e-06,2312.853261,...,-4.651338,2.769356,-0.527245,-1.024308,2.630192,-2.558457,-4.056424,6.664631,7.100372,99180-9-0-49.wav


: 